In [21]:
import pixiedust
pixiedust.installPackage("org.apache.bahir:spark-sql-cloudant_2.11:0")

PackageManager is disabled because /home/spark/shared/user-libs/spark2/ is not on the Java Class Path. Please contact your administrator


In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [23]:
def readDataFrameFromCloudant(database):

    cloudantdata = spark.read.format("org.apache.bahir.cloudant")\
        .option("cloudant.host",'22481d5a-1499-4599-a057-1a50548b54f8-bluemix.cloudantnosqldb.appdomain.cloud')\
        .option("cloudant.username", '22481d5a-1499-4599-a057-1a50548b54f8-bluemix')\
        .option("cloudant.password",'ee3114eff8372f6bada2b226cbad88291b950ebd05336bcd26f3664363c1a117')\
        .load(database)

    return cloudantdata

In [24]:
df = readDataFrameFromCloudant('training')
#df.show()
df.createOrReplaceTempView('df')
spark.sql('select class, count(class) from df group by class').show()

+-----+------------+
|class|count(class)|
+-----+------------+
|    0|          56|
|    1|          59|
+-----+------------+



In [25]:
display(df)

_id,_rev,ax,ay,az,class,time
0417401354059fa89cd604107b6d3bea,1-5417d1f32b7bc53f7ff3e1d8cb7cd6e8,-0.20273379981517792,-0.4499572813510895,10.122535705566406,0,1605562047795
0417401354059fa89cd604107b6d4827,1-2673c042698f6c058828a2150ab5a41a,-0.2007574886083603,-0.4576183259487152,10.102157592773438,0,1605562053238
0417401354059fa89cd604107b6d534e,1-38e4063506beb5417cbbdad226b0bb9f,-0.2000342607498169,-0.4524599015712738,10.10208511352539,0,1605562065075
0417401354059fa89cd604107b6d5c9b,1-edcc339a2e4a6c58d5730dd0f4f61cd2,16.319236755371094,-15.684097290039062,14.222559928894043,1,1605562126048
0417401354059fa89cd604107b6d6325,1-703593062fade8e0ac56b3b4c55b5ed0,3.888608932495117,6.443172931671143,-1.6969505548477173,1,1605562138046
0417401354059fa89cd604107b6d69f6,1-f6f6b9a6139bee9b996e48417d63211d,-6.517452239990234,7.163028717041016,-9.663651466369629,1,1605562140212
0417401354059fa89cd604107b6d7720,1-0328244c6538ade6fd1f072a7f814692,8.367769241333008,0.3984348475933075,-8.927811622619629,1,1605562152164
0417401354059fa89cd604107b6d7fea,1-b8404983cd41c0050bc1c25e800bcfef,-8.000436782836914,-0.09563558548688889,-6.315465450286865,1,1605562161852
23257c063457a5daa687f081a26a6f5e,1-e563d83790cdff67efeb79e950f537f3,-0.20423533022403717,-0.45521625876426697,10.106805801391602,0,1605562008634
23257c063457a5daa687f081a26a787d,1-00047c26cb678705ad1d0f7e1403fe12,-0.20608389377593994,-0.4578677713871002,10.111226081848145,0,1605562011951


In [26]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [29]:
vectorAssembler = VectorAssembler(inputCols=["ax","ay","az"], outputCol="features")
lr = LogisticRegression(maxIter=1000).setLabelCol("class")
pipeline = Pipeline(stages=[vectorAssembler, lr ])
model = pipeline.fit(df)
result = model.transform(df)

In [32]:
print('intercept = ', model.stages[1].intercept)
print('coefficients = ', model.stages[1].coefficients)

intercept =  63.632019087063675
coefficients =  [1.8503643184625316,-0.1161077261952959,-6.6259155578140225]


In [30]:
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy").setPredictionCol("prediction").setLabelCol("class")
binEval.evaluate(result)

0.991304347826087

In [31]:
new_df = readDataFrameFromCloudant('training')
new_df.createOrReplaceTempView('new_df')
result = model.transform(new_df)
result.createOrReplaceTempView('result')
spark.sql("select ax, ay, az, class, prediction from result").show(50)

+--------------------+--------------------+-------------------+-----+----------+
|                  ax|                  ay|                 az|class|prediction|
+--------------------+--------------------+-------------------+-----+----------+
|-0.20273379981517792| -0.4499572813510895| 10.122535705566406|    0|       0.0|
| -0.2007574886083603| -0.4576183259487152| 10.102157592773438|    0|       0.0|
| -0.2000342607498169| -0.4524599015712738|  10.10208511352539|    0|       0.0|
|  16.319236755371094| -15.684097290039062| 14.222559928894043|    1|       1.0|
|   3.888608932495117|   6.443172931671143|-1.6969505548477173|    1|       1.0|
|  -6.517452239990234|   7.163028717041016| -9.663651466369629|    1|       1.0|
|   8.367769241333008|  0.3984348475933075| -8.927811622619629|    1|       1.0|
|  -8.000436782836914|-0.09563558548688889| -6.315465450286865|    1|       1.0|
|-0.20423533022403717|-0.45521625876426697| 10.106805801391602|    0|       0.0|
|-0.20608389377593994| -0.45